In [1]:
import pandas as pd
import numpy as np
from xlsxwriter.utility import xl_rowcol_to_cell

In [2]:
df = pd.read_excel("AdvWorksData.xlsx")

In [ ]:
df.head()

In [3]:
df = df[['productcategory','productsubcategory','product','Country','OrderDate','StandardCost','Discount','Sales']]

In [4]:
df.rename(columns={"productcategory": "Category","productsubcategory": "Sub-Category","product":"Product","StandardCost": "Cost"}, inplace=True)

In [5]:
df = df[(df['Category']=='Clothing') & (df['Country']=='United States') & (df['Product']=='Classic Vest, M')]

In [6]:
df= df[ (df['OrderDate'] == '2014-05-01')]

In [7]:
df.shape

(15, 8)

In [8]:
#add a new column for Net Sales
df['Net'] = df['Sales'] - (df['Cost']-df['Discount'])
#convert datetime to date and string
df['OrderDate'] = df['OrderDate'].astype(str)

In [9]:
df.head()

,Category,Sub-Category,Product,Country,OrderDate,Cost,Discount,Sales,Net
2736,Clothing,Vests,"Classic Vest, M",United States,2014-05-01,23.749,0.0,152.4,128.651
20398,Clothing,Vests,"Classic Vest, M",United States,2014-05-01,23.749,0.0,152.4,128.651
20399,Clothing,Vests,"Classic Vest, M",United States,2014-05-01,23.749,0.0,152.4,128.651
20400,Clothing,Vests,"Classic Vest, M",United States,2014-05-01,23.749,0.0,38.1,14.351
20401,Clothing,Vests,"Classic Vest, M",United States,2014-05-01,23.749,0.0,76.2,52.451


In [ ]:
df['OrderDate'] = df['OrderDate'].astype(str)

### Simple Outupt

In [10]:
#simpel output
df.to_excel('simple.xlsx', index=False, sheet_name='report')

### Get access to worksheet object

In [11]:
writer = pd.ExcelWriter('enhanced.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=False, sheet_name='report')
#
workbook = writer.book
worksheet = writer.sheets['report']

In [12]:
#Now we have the worksheet object. We can manipulate it 
worksheet.set_zoom(90)

In [13]:
#Set header formating
header_format = workbook.add_format({
        "valign": "vcenter",
        "align": "center",
        "bg_color": "#951F06",
         "bold": True,
        'font_color': '#FFFFFF'
    })

In [14]:
#add title
title = "Monthly Sales Report "
#merge cells
format = workbook.add_format()
format.set_font_size(20)
format.set_font_color("#333333")
#
subheader = "Sales report for Classic Vest, M"
worksheet.merge_range('A1:AS1', title, format)
worksheet.merge_range('A2:AS2', subheader)
worksheet.set_row(2, 15) # Set the header row height to 15
# puting it all together
# Write the column headers with the defined format.
for col_num, value in enumerate(df.columns.values):
    #print(col_num, value)
    worksheet.write(2, col_num, value, header_format)

In [15]:
# Add a number format for cells with money.
money_fmt = workbook.add_format({'num_format': '$#,##0.00'})

In [16]:
# Total formatting
total_fmt = workbook.add_format({'align': 'right', 'num_format': '$#,##0',
                                 'bold': True, 'bottom':6})

In [17]:
# Adjust the column width.
worksheet.set_column('A:F', 20)

0

In [18]:
# numeric columns
worksheet.set_column('F:I', 12, money_fmt)

0

In [19]:
number_rows = len(df.index)

In [20]:
# Add total rows
for column in range(5, 9):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(number_rows+1, column)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(3, column)
    print(start_range)
    end_range = xl_rowcol_to_cell(number_rows, column)
    print(end_range)
    # Construct and write the formula
    formula = "=SUM({:s}:{:s})".format(start_range, end_range)
    print(formula)
    worksheet.write_formula(cell_location, formula, total_fmt)

F4
F16
=SUM(F4:F16)
G4
G16
=SUM(G4:G16)
H4
H16
=SUM(H4:H16)
I4
I16
=SUM(I4:I16)


In [21]:
# Add a total label
worksheet.write_string(number_rows+1, 4, "Total",total_fmt)

0

### Advance Outuput

In [22]:
#Advance output
writer.save()